In [1]:
import numpy as np

import torch

from tqdm import tqdm
import copy

import sys, os

import json

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
base_path = '/Users/alberic/Desktop/divers/projects/hvac_opt'
file = 'combined_Room1'
with open(os.path.join(base_path, 'weather_model', 'data', file, 'normalization.json')) as f:
    normalization_data = json.load(f)

In [13]:
from regressive_model import MLPModel
import torch
past_window = 16
model = MLPModel(d_input=29*past_window, d_output=2, d_layers=[256, 128, 64, 32, 16, 8])
model.load_state_dict(torch.load('/Users/alberic/Desktop/divers/projects/hvac_opt/weather_model/saved_models/12.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [15]:
## Testing loop ##

future_window = 64
test_data = np.load(os.path.join('/Users/alberic/Desktop/divers/projects/hvac_opt/weather_model/data/combined_Room1', 
                                    f'normalized_data_{1}.npy')) 
    
time = test_data[:,0]*(12*31*24) + test_data[:,1]*(31*24) + test_data[:,2]*(24) + test_data[:,3]
time -= time[0]

def denormalize(output):

    output = copy.copy(output)

    output[:,0] *= normalization_data['humidity'][1]
    output[:,0] += normalization_data['humidity'][0]
    
    output[:,1] *= normalization_data['temperature'][1]
    output[:,1] += normalization_data['temperature'][0]

    return output

rmse_future = np.zeros(2)

fig = make_subplots(rows=2, cols=1)

# fig.add_trace(go.Scatter(x=np.linspace(0,max_epochs, len(model.train_losses)) ,y=model.train_losses, name='Loss'), row=3, col=1)
# fig.add_trace(go.Scatter(y=model.valid_losses, name='Loss'), row=3, col=1)

test_data_denorm = denormalize(test_data[:, 6:8])
fig.add_trace(go.Scatter(x=time, y=test_data_denorm[:,0], name='Humidity'), row=1, col=1)
fig.add_trace(go.Scatter(x=time, y=test_data_denorm[:,1], name='Temperature'), row=2, col=1)


for i in tqdm(range(0, len(test_data)-future_window-(past_window-1))):

    input = copy.copy(test_data[i:i+past_window].flatten())
    outputs = np.zeros((future_window, 2))
    # outputs[0] = input[6:8]
    for j in range(future_window):

        with torch.no_grad():
            outputs[j] = model(torch.tensor(input).unsqueeze(0).type(torch.float))[0]
            # print(torch.tensor(input).type(torch.float))

            if i+past_window+j<len(test_data):
                input[:-29] = input[29:]
                input[-29:] = copy.copy(test_data[i+j+past_window])
                input[-29+6:-29+8] = outputs[j]

    rmse_future += np.mean(np.sqrt((denormalize(outputs) - test_data_denorm[i+past_window:i+past_window+future_window])**2), axis=0)

    if i%future_window==0:
        # outputs = np.concatenate((test_data[i:i+1,6:8], outputs))
        outputs_denorm = denormalize(outputs)
        color = ['orange', 'green'][int((i/future_window)%2)]
        fig.add_trace(go.Scatter(x=time[i+past_window:i+past_window+future_window+0], y=outputs_denorm[:, 0], opacity=0.5, marker_color=color, showlegend=False), row=1, col=1)
        fig.add_trace(go.Scatter(x=time[i+past_window:i+past_window+future_window+0], y=outputs_denorm[:, 1], opacity=0.5, marker_color=color, showlegend=False), row=2, col=1)

rmse_future /= len(test_data)-future_window


fig.update_layout(title=f'Temperature and humidity over time [hr], rmse over one hour: humidity={rmse_future[0]:.3f} and temperature={rmse_future[1]:.3f}',
                    height=1000)

fig.show()

100%|██████████| 209/209 [00:01<00:00, 159.40it/s]


In [13]:
0.98**100

0.13261955589475294